In [38]:

from copy import deepcopy
from sklearn.linear_model import LogisticRegression
import category_encoders as ce    
import pickle
import numpy as np

def project2_scoring_func(data):
    
    X = data.copy()
    
    '''Load Artifacts'''
    
    artifacts_dict_file = open("artifacts_dictnew_file.pkl", "rb")
    artifacts_dict = pickle.load(file=artifacts_dict_file)
    artifacts_dict_file.close()

    col_remove_dollar = artifacts_dict["col_remove_dollar"]
    cat_encoders = artifacts_dict["cat_encoders"]
    model = artifacts_dict["model"]  
    scoring_columns = artifacts_dict["scoring_columns"]
    threshold = artifacts_dict['threshold']
   
    values_to_fill = {}
    for col in X.columns:
        if X[col].isna().any() == True:
            if pd.api.types.is_numeric_dtype(X[col].dtype):
                values_to_fill[col] = 0
            else:
                values_to_fill[col] = "Missing"
    X.fillna(value=values_to_fill,inplace=True)
   
    
    # Convert specific columns to string type
    for col in col_remove_dollar:
        X[col] = X[col].astype(str)
    
    # Remove Dollars from numerical vars
    for col in col_remove_dollar:
        X[col] = X[col].str.replace('$','',regex=True).replace(',','',regex=True)
        X[col] = X[col].astype(float)
    
    import numpy as np

    # whether the bank and customer are in the same state
    X['state_and_bankstate'] = np.where(X['State'] == 'BankState', 1, 0)

    
    # The difference between initial approved amount and final amount of loan
    X['diff_loan_amt1'] = X['GrAppv'] - X['SBA_Appv']
    
    # The difference between approved amount of loan and the requested amount of loan
    X['diff_loan_amt2'] = X['DisbursementGross'] - X['SBA_Appv']
    
    # amount of installment
    X['total_job'] = X['CreateJob'] * X['RetainedJob']
    
    #The ratio of the gross amount of loan approved by the bank (GrAppv) to the requested amount of loan (DisbursementGross).
    X['loan_approval_rate'] = X['GrAppv'] / X['DisbursementGross']
    
    #The ratio of the number of business employees (NoEmp) to the gross amount of the loan approved by the bank (GrAppv).
    X['employees_per_loan_amount'] = X['NoEmp'] / X['GrAppv']
    
    #The ratio of the loan amount approved by the SBA (SBA_Appv) to the requested amount of loan (DisbursementGross).
    X['sba_approval_rate'] = X['SBA_Appv'] / X['DisbursementGross']
    
    #indicates whether the business is a franchise or not based on the FranchiseCode column
    X['franchise_or_not'] = np.where(X['FranchiseCode'] <= 1, 0, 1)
    
    # the ratio of the gross amount of the loan approved by the bank (GrAppv) to the number of business employees (NoEmp)
    X['loan_amount_per_employee'] = X['GrAppv'] / X['NoEmp']
    
    #the ratio of the outstanding balance (BalanceGross) to the gross amount of the loan approved by the bank (GrAppv).
    X['loan_balance_ratio'] = X['BalanceGross'] / X['GrAppv']

  
    # -- Drop NewExist = 0
    index_exist0 = []
    for item in list(X[X['NewExist'] == 0].index):
        index_exist0.append(item)

    X.drop(index_exist0, inplace=True)

        # -- Change newExist
    def change_newexist(x):
        if x == 1:
            return 1
        else:
            return 0
    X['NewExist'] = X['NewExist'].apply(lambda x : change_newexist(x))
    # -- change franchise

    def change_franchise(x):
        if (x == 1) | (x == 0):
            return 0
        else :
            return 1

    X['FranchiseCode'] = X['FranchiseCode'].apply(lambda x: change_franchise(x))
    #dropping values other than Y or N for LowDoc
    index_drop_lowdoc = []
    for index in list(X[(X['LowDoc'] != 'N') & (X['LowDoc'] != 'Y')].index):
        index_drop_lowdoc.append(index)

    X.drop(index_drop_lowdoc,inplace=True)
    
    #dropping values other than Y, N ,0,T
    index_drop_RevLineCr = []
    for index in list(X[(X['RevLineCr'] != 'N') & (X['RevLineCr'] != 'Y') & (X['RevLineCr'] != '0') & (X['RevLineCr'] != 'T')].index):
        index_drop_RevLineCr.append(index)

    X.drop(index_drop_RevLineCr, inplace=True)


  
    from sklearn.preprocessing import OneHotEncoder
    from category_encoders import TargetEncoder
    from copy import deepcopy

    '''Save original columns that need to be droped or not used
      Save One-hot and Label encoders for future use
    '''
       # Categorical encoding
    # Columns to drop from ML models
    cols_to_drop = ["City", "State", "Bank", "BankState", "RevLineCr"]
    # Categorical encoders dictionary
    cat_encoders = {}
    # New categorical (encoded) columns
    cat_enc_columns = []

    for col, (encoder, enc_type) in cat_encoders.items():
        if col not in ["City", "State", "Bank", "BankState", "RevLineCr"]:
            if enc_type == "ohe":
                ohe = encoder
                result = ohe.transform(X[[col]])
                ohe_columns = [col + "_" + str(x) for x in ohe.categories_[0]]
                cat_enc_columns = cat_enc_columns + ohe_columns
                new_encoded = pd.DataFrame(result, columns=ohe_columns, index=X.index)
            elif enc_type == "trg":
                trg = encoder
                new_col_name = col + "_trg"
                result = trg.transform(X[[col]])
                new_encoded = pd.DataFrame(result, columns=[new_col_name], index=X.index)
        X=pd.concat([X, new_encoded.reindex(X.index)], axis=1, join="inner")

    X.drop(cols_to_drop, axis=1, inplace=True)
    
    
      
    # light gbm predict
    scoring_columns=artifacts_dict['scoring_columns']
    X_test=scoring_columns
    model=artifacts_dict['model']
    y_pred_proba = model.predict(X_test)
    y_pred_binary = np.round(y_pred_proba)
    y_pred_binary=y_pred_binary.astype(int)
    
    
    df = {"label":y_pred,
         "probability_0":y_pred_binary[0],
         "probability_1":y_pred_binary[1]}
    
    return pd.DataFrame(df)
    
    

In [39]:

import pandas as pd

# Read in the CSV file into a DataFrame
df = pd.read_csv('E:\machine_learning\data\SBA_loans_project_2.zip')

# Drop the 'MIS_Status' and 'index' columns from the DataFrame
df = df.drop(columns=['MIS_Status', 'index'])

# Sample 10,000 random rows from the DataFrame to create a new DataFrame 'test_data'
test_data = df.sample(n=10000, random_state=234)

# Reset the index of the 'test_data' DataFrame
test_data = test_data.reset_index(drop=True)


print(test_data)

                                City State    Zip  \
0                      IRON MOUNTAIN    MI  49801   
1                     CORONA DEL MAR    CA  92625   
2                          ENGLEWOOD    NJ   7631   
3                        LOS ANGELES    CA  90032   
4                          Annapolis    MD  21409   
...                              ...   ...    ...   
9995                    Indianapolis    IN  46227   
9996  BOISE (CORPORATE NAME BOISE CI    ID  83705   
9997                    GRAND RAPIDS    MI  49512   
9998                        BRILLION    WI  54110   
9999                   WILLIAMSVILLE    NY  14221   

                               Bank BankState   NAICS  NoEmp  NewExist  \
0       WELLS FARGO BANK NATL ASSOC        SD       0      1       2.0   
1         CDC SMALL BUS. FINAN CORP        CA       0     16       1.0   
2            CAPITAL ONE NATL ASSOC        VA  561990      7       1.0   
3                         BBCN BANK        CA  442299      2       

In [ ]:
#To test scoring function
project2_scoring_func(test_data)